In [1]:
library(repr)
library(tidyverse)
library(repr)
library(tidymodels)
library(dplyr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [3]:
players <- read_csv("data/players.csv")
head(players)

Rows: 196 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, age
lgl (3): subscribe, individualId, organizationName

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,NA,NA
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17,NA,NA
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17,NA,NA
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21,NA,NA
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21,NA,NA
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17,NA,NA


In [13]:
players_renamed <- players |>
mutate(experience = recode(experience, "Beginner" = 1, "Amateur"=2, "Regular"=3, "Pro"=4, "Veteran"=5)) |>
mutate(subscribe = recode(as.character(subscribe), "FALSE"=0, "TRUE"=1))

#I think more than 50 hours is an outlier
count(players_renamed, played_hours > 50) 


played_hours > 50,n
<lgl>,<int>
FALSE,190
TRUE,6


There's only 6 players that played more than 50, a few of them were playing 150 to 200 hours which indicates there's noise in the data

In [14]:

players_renamed <- subset(players_renamed,played_hours < 50)
head(players_renamed)


experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
4,1,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,NA,NA
5,1,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17,NA,NA
5,0,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17,NA,NA
2,1,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21,NA,NA
3,1,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21,NA,NA
2,1,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17,NA,NA


In [15]:
set.seed(999)
players_split2 <- initial_split(players_renamed, prop=0.7, strata = played_hours)
players_training2 <-training(players_split2)
players_testing2 <-testing(players_split2)

In [16]:
lm_spec <- linear_reg() |>
    set_engine('lm') |>
    set_mode('regression')

lm_recipe <- recipe(played_hours ~ subscribe, data =players_training2) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

lm_fit <- workflow() |>
            add_recipe(lm_recipe) |>
            add_model(lm_spec) |>
            fit(data = players_training2)



In [17]:
lm_test_results <- lm_fit |>
                    predict(players_testing2) |>
                    bind_cols(players_testing2) |>
                    metrics(truth = played_hours, estimate = .pred)       

lm_rmspe <- lm_test_results |>
                filter(.metric == 'rmse') |>
                select(.estimate) |>
                pull()
lm_rmspe

[1] 2.896639